In [18]:
import numpy as np

from mitiq import cdr, Observable, PauliString, Executor

import cirq
from cirq.contrib.qasm_import import circuit_from_qasm
from cirq.circuits.qasm_output import QasmUGate

from qiskit import *
from qiskit import transpile, assemble, IBMQ
from qiskit.visualization import *
from qiskit  import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector

# VNCDR on Feature Map

In [34]:
pi = np.pi
feature_dim = 2
reps = 2
duplicates = 1
feature_map_circuit = QuantumCircuit(feature_dim, name='feature_map_circuit')
# x = ParameterVector('x', feature_dim * reps)
x = [0.1, 0.2]

for i in range(reps):
    for i in range(feature_dim):
        feature_map_circuit.h(i)

    feature_map_circuit.rz(2.0*x[0], 0)
    feature_map_circuit.rz(2.0*x[1], 1)
    feature_map_circuit.cnot(0, 1)
    feature_map_circuit.rz(2.0*(pi - x[0])*(pi - x[1]), 1)
    feature_map_circuit.cnot(0, 1)

feature_map_circuit.draw()

┌───┐┌─────────┐                        ┌───┐┌─────────┐     »
q_0: ┤ H ├┤ Rz(0.2) ├──■──────────────────■──┤ H ├┤ Rz(0.2) ├──■──»
     ├───┤├─────────┤┌─┴─┐┌────────────┐┌─┴─┐├───┤├─────────┤┌─┴─┐»
q_1: ┤ H ├┤ Rz(0.4) ├┤ X ├┤ Rz(17.894) ├┤ X ├┤ H ├┤ Rz(0.4) ├┤ X ├»
     └───┘└─────────┘└───┘└────────────┘└───┘└───┘└─────────┘└───┘»
«                        
«q_0: ────────────────■──
«     ┌────────────┐┌─┴─┐
«q_1: ┤ Rz(17.894) ├┤ X ├
«     └────────────┘└───┘

In [32]:
qasm_feature_map = feature_map_circuit.qasm(filename='feature_map.qasm')
print(qasm_feature_map)
cirq_feature_map = circuit_from_qasm(qasm_feature_map)
cirq_3_feature_map = cirq_feature_map * 3
print(cirq_3_feature_map)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
h q[0];
h q[1];
rz(0.2) q[0];
rz(0.4) q[1];
cx q[0],q[1];
rz(17.894253) q[1];
cx q[0],q[1];
h q[0];
h q[1];
rz(0.2) q[0];
rz(0.4) q[1];
cx q[0],q[1];
rz(17.894253) q[1];
cx q[0],q[1];

q_0: ───H───Rz(0.064π)───@──────────────@───H───Rz(0.064π)───@──────────────@───H───Rz(0.064π)───@──────────────@───H───Rz(0.064π)───@──────────────@───H───Rz(0.064π)───@──────────────@───H───Rz(0.064π)───@──────────────@───
                         │              │                    │              │                    │              │                    │              │                    │              │                    │              │
q_1: ───H───Rz(0.127π)───X───Rz(1.7π)───X───H───Rz(0.127π)───X───Rz(1.7π)───X───H───Rz(0.127π)───X───Rz(1.7π)───X───H───Rz(0.127π)───X───Rz(1.7π)───X───H───Rz(0.127π)───X───Rz(1.7π)───X───H───Rz(0.127π)───X───Rz(1.7π)───X───


In [26]:
from mitiq.interface.mitiq_cirq import compute_density_matrix

compute_density_matrix(cirq_feature_map).round(3)


array([[ 0.742+0.j   , -0.03 +0.138j,  0.02 +0.109j, -0.207-0.289j],
       [-0.03 -0.138j,  0.044-0.j   ,  0.02 -0.006j, -0.043+0.052j],
       [ 0.02 -0.109j,  0.02 +0.006j,  0.031-0.j   , -0.047+0.023j],
       [-0.207+0.289j, -0.043-0.052j, -0.047-0.023j,  0.184+0.j   ]],
      dtype=complex64)

In [27]:
obs = Observable(PauliString("ZZ"))
print(obs)

Z(q(0))*Z(q(1))


In [28]:
def simulate(circuit: cirq.Circuit) -> np.ndarray:
    return compute_density_matrix(circuit, noise_level=(0.0,))

simulate(cirq_feature_map).round(3)

array([[ 0.75 +0.j   , -0.045+0.15j ,  0.007+0.109j, -0.235-0.309j],
       [-0.045-0.15j ,  0.033-0.j   ,  0.021-0.008j, -0.048+0.066j],
       [ 0.007-0.109j,  0.021+0.008j,  0.016+0.j   , -0.047+0.031j],
       [-0.235+0.309j, -0.048-0.066j, -0.047-0.031j,  0.201-0.j   ]],
      dtype=complex64)

In [29]:
ideal_measurement = obs.expectation(cirq_feature_map, simulate).real
print("ideal_measurement = ",ideal_measurement)

ideal_measurement =  0.9027009084820747


In [30]:
unmitigated_measurement = obs.expectation(cirq_feature_map, compute_density_matrix).real
print("unmitigated_measurement = ", unmitigated_measurement)

unmitigated_measurement =  0.8512219544500113


In [31]:
mitigated_measurement = cdr.execute_with_cdr(
    cirq_feature_map,
    compute_density_matrix,
    observable=obs,
    simulator=simulate,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)


/Users/Shared/anaconda3/envs/mitiq-env/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


mitigated_measurement =  0.9447179995878725


In [36]:
from mitiq.zne import scaling

vncdr = cdr.execute_with_cdr(
    cirq_feature_map,
    compute_density_matrix,
    observable=obs,
    simulator=simulate,
    seed=0,
    scale_factors=(1.0, 3.0),
).real

print("vncdr = ", vncdr)


/Users/Shared/anaconda3/envs/mitiq-env/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


vncdr =  0.9026062206502856


In [37]:
error_unmitigated = abs(unmitigated_measurement-ideal_measurement)
error_mitigated = abs(mitigated_measurement-ideal_measurement)
error_vn_mitigated = abs(vncdr-ideal_measurement)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated with CDR):", error_mitigated)
print("Error (mitigated with VNCDR):", error_vn_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):",
      error_mitigated/ideal_measurement)
print("Relative error (mitigated with VNCDR):",
      error_vn_mitigated/ideal_measurement)

print(
    f"Error reduction with CDR: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")
print(
    f"Error reduction with VNCDR: {(error_unmitigated-error_vn_mitigated)/error_unmitigated :.1%}.")


Error (unmitigated): 0.051478954032063484
Error (mitigated with CDR): 0.042017091105797766
Error (mitigated with VNCDR): 9.468783178911977e-05
Relative error (unmitigated): 0.05702769715677728
Relative error (mitigated with CDR): 0.046545971883922295
Relative error (mitigated with VNCDR): 0.00010489391436233391
Error reduction with CDR: 18.4%.
Error reduction with VNCDR: 99.8%.


# VNCDR on Ansatz

In [38]:
num_qubits = 2
ansatz_circuit = QuantumCircuit(num_qubits, name='ansatz_circuit')
reps = 1
# params = ParameterVector('θ', num_qubits + num_qubits * reps)
params = [0.7, 0.3, 0.5, 1.2]
counter = 0

for i in range(num_qubits):
    ansatz_circuit.ry(params[counter], i)
    counter += 1

for j in range(reps):
    for k in range(num_qubits-1):
        ansatz_circuit.cnot(k, k+1)
    for m in range(num_qubits):
        ansatz_circuit.ry(params[counter], m)
        counter += 1

ansatz_circuit.draw()

┌─────────┐     ┌─────────┐
q_0: ┤ Ry(0.7) ├──■──┤ Ry(0.5) ├
     ├─────────┤┌─┴─┐├─────────┤
q_1: ┤ Ry(0.3) ├┤ X ├┤ Ry(1.2) ├
     └─────────┘└───┘└─────────┘

In [58]:
qasm_ansatz = ansatz_circuit.qasm(filename='ansatz.qasm')
print(qasm_ansatz)
cirq_ansatz = circuit_from_qasm(qasm_ansatz) * 1
print(cirq_ansatz)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
ry(0.7) q[0];
ry(0.3) q[1];
cx q[0],q[1];
ry(0.5) q[0];
ry(1.2) q[1];

q_0: ───Ry(0.223π)───@───Ry(0.159π)───
                     │
q_1: ───Ry(0.095π)───X───Ry(0.382π)───


In [59]:
compute_density_matrix(cirq_ansatz).round(3)

array([[0.498+0.j, 0.381+0.j, 0.022+0.j, 0.315+0.j],
       [0.381+0.j, 0.296+0.j, 0.015+0.j, 0.243+0.j],
       [0.022+0.j, 0.015+0.j, 0.004+0.j, 0.013+0.j],
       [0.315+0.j, 0.243+0.j, 0.013+0.j, 0.201+0.j]], dtype=complex64)

In [60]:
simulate(cirq_ansatz).round(3)

array([[0.494+0.j, 0.382+0.j, 0.018+0.j, 0.322+0.j],
       [0.382+0.j, 0.296+0.j, 0.014+0.j, 0.249+0.j],
       [0.018+0.j, 0.014+0.j, 0.001+0.j, 0.012+0.j],
       [0.322+0.j, 0.249+0.j, 0.012+0.j, 0.209+0.j]], dtype=complex64)

In [61]:
ideal_measurement = obs.expectation(cirq_ansatz, simulate).real
print("ideal_measurement = ",ideal_measurement)

ideal_measurement =  0.40678397472947836


In [62]:
unmitigated_measurement = obs.expectation(cirq_ansatz, compute_density_matrix).real
print("unmitigated_measurement = ", unmitigated_measurement)

unmitigated_measurement =  0.3995709279552102


In [63]:
mitigated_measurement = cdr.execute_with_cdr(
    cirq_ansatz,
    compute_density_matrix,
    observable=obs,
    simulator=simulate,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)


mitigated_measurement =  2.801286977714698


/Users/Shared/anaconda3/envs/mitiq-env/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [64]:
from mitiq.zne import scaling

vncdr = cdr.execute_with_cdr(
    cirq_ansatz,
    compute_density_matrix,
    observable=obs,
    simulator=simulate,
    seed=0,
    scale_factors=(1.0, 3.0),
).real

print("vncdr = ", vncdr)


vncdr =  -0.833781183231622


/Users/Shared/anaconda3/envs/mitiq-env/lib/python3.8/site-packages/numpy/core/_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Users/Shared/anaconda3/envs/mitiq-env/lib/python3.8/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [65]:
error_unmitigated = abs(unmitigated_measurement-ideal_measurement)
error_mitigated = abs(mitigated_measurement-ideal_measurement)
error_vn_mitigated = abs(vncdr-ideal_measurement)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated with CDR):", error_mitigated)
print("Error (mitigated with VNCDR):", error_vn_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):",
      error_mitigated/ideal_measurement)
print("Relative error (mitigated with VNCDR):",
      error_vn_mitigated/ideal_measurement)

print(
    f"Error reduction with CDR: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")
print(
    f"Error reduction with VNCDR: {(error_unmitigated-error_vn_mitigated)/error_unmitigated :.1%}.")


Error (unmitigated): 0.00721304677426815
Error (mitigated with CDR): 2.3945030029852195
Error (mitigated with VNCDR): 1.2405651579611003
Relative error (unmitigated): 0.017731885281530593
Relative error (mitigated with CDR): 5.886424126165798
Relative error (mitigated with VNCDR): 3.0496903394144463
Error reduction with CDR: -33096.8%.
Error reduction with VNCDR: -17098.9%.


- Clearly VNCDR / CDR doesn't work well on low-depth circuits.